In [1]:
import requests, base64
from dotenv import load_dotenv
import os
from fastapi.responses import RedirectResponse

load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
SPOTIFY_REDIRECT_URI = os.getenv("SPOTIFY_REDIRECT_URI")
SPOTIFY_SCOPES = os.getenv("SPOTIFY_SCOPES")


In [3]:
AUTH_URL = auth_url = (
        "https://accounts.spotify.com/authorize"
        f"?client_id={SPOTIFY_CLIENT_ID}"
        f"&response_type=code"
        f"&redirect_uri={SPOTIFY_REDIRECT_URI}"
        f"&scope={SPOTIFY_SCOPES}"
    )



In [4]:
redirect_auth = RedirectResponse(url=AUTH_URL)

In [5]:
print(redirect_auth)


In [6]:
from fastapi import FastAPI, Request
from fastapi.responses import RedirectResponse
import requests, base64

app = FastAPI()

CLIENT_ID = SPOTIFY_CLIENT_ID
CLIENT_SECRET = SPOTIFY_CLIENT_SECRET
REDIRECT_URI = SPOTIFY_REDIRECT_URI
SCOPE = SPOTIFY_SCOPES

@app.get("/")
def login():
    auth_url = (
        "https://accounts.spotify.com/authorize"
        f"?client_id={CLIENT_ID}"
        f"&response_type=code"
        f"&redirect_uri={REDIRECT_URI}"
        f"&scope={SCOPE}"
    )
    return RedirectResponse(auth_url)

@app.get("/callback")
def callback(request: Request):
    code = request.query_params.get("code")
    token_url = "https://accounts.spotify.com/api/token"

    auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
    headers = {"Authorization": f"Basic {auth_header}"}
    data = {
        "grant_type": "authorization_code",
        "code": code,
        "redirect_uri": REDIRECT_URI
    }

    r = requests.post(token_url, headers=headers, data=data)
    token_info = r.json()

    access_token = token_info.get("access_token")
    refresh_token = token_info.get("refresh_token")

    return {
        "access_token": access_token,
        "refresh_token": refresh_token
    }

@app.get("/me")
def get_user_profile(access_token: str):
    headers = {"Authorization": f"Bearer {access_token}"}
    r = requests.get("https://api.spotify.com/v1/me", headers=headers)
    return r.json()


In [ ]:
import requests
import base64
from urllib.parse import urlencode
from http.server import HTTPServer, BaseHTTPRequestHandler
import webbrowser

CLIENT_ID = SPOTIFY_CLIENT_ID
CLIENT_SECRET = SPOTIFY_CLIENT_SECRET
REDIRECT_URI = SPOTIFY_REDIRECT_URI
SCOPE = SPOTIFY_SCOPES

# Step 1: Ask user to log in
params = {
    "client_id": CLIENT_ID,
    "response_type": "code",
    "redirect_uri": REDIRECT_URI,
    "scope": SCOPE,
}
auth_url = f"https://accounts.spotify.com/authorize?{urlencode(params)}"
print("Opening Spotify login URL...")
webbrowser.open(auth_url)

# Step 2: Simple local HTTP server to catch redirect
code_container = {}

class Handler(BaseHTTPRequestHandler):
    def do_GET(self):
        if "/callback" in self.path:
            from urllib.parse import parse_qs, urlparse
            query = parse_qs(urlparse(self.path).query)
            code_container["code"] = query.get("code", [None])[0]
            self.send_response(200)
            self.end_headers()
            self.wfile.write(b"You can close this window now.")
        else:
            self.send_response(404)
            self.end_headers()

server = HTTPServer(("localhost", 8080), Handler)
print("Waiting for Spotify authorization...")
server.handle_request()
server.server_close()

auth_code = code_container.get("code")
print("Authorization Code:", auth_code)

# Step 3: Exchange code for access + refresh token
token_url = "https://accounts.spotify.com/api/token"
auth_header = base64.b64encode(f"{CLIENT_ID}:{CLIENT_SECRET}".encode()).decode()
headers = {"Authorization": f"Basic {auth_header}"}
data = {
    "grant_type": "authorization_code",
    "code": auth_code,
    "redirect_uri": REDIRECT_URI,
}

r = requests.post(token_url, headers=headers, data=data)
tokens = r.json()
print("Access Token:", tokens.get("access_token"))
print("Refresh Token:", tokens.get("refresh_token"))

# Step 4: Example API call
headers = {"Authorization": f"Bearer {tokens.get('access_token')}"}
me = requests.get("https://api.spotify.com/v1/me", headers=headers).json()
print(me)


Opening Spotify login URL...
Waiting for Spotify authorization...
